In [18]:
'''
The classic and simple LSTM on keras example, from the official repository.
~~Train a recurrent convolutional network on the IMDB sentiment classification task.~~
~~Baseline: gets to 0.8498 test accuracy after 2 epochs. 41 s/epoch on K520 GPU.~~
~~batch_size is highly sensitive.~~
~~Only 2 epochs are needed as the dataset is very small.~~
'''

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding
from keras.layers import LSTM, Conv1D, MaxPooling1D
from keras.datasets import imdb
from keras.callbacks import Callback, EarlyStopping, ModelCheckpoint
import tensorflow as tf
import numpy as np
import random

## PARAMETERS ##
print_tensorflow_GPU_info = True
tensorflow_verbosity = "INFO"  # DEBUG(10): All | INFO(20): Info&Warning | WARN(30)[Default]: Warning | ERROR(40): Error | FATAL(50): None
random_state = 22
    # Embedding
max_features = 20000
maxlen = 100
embedding_size = 128
dropout = 0.25
    # Convolution
kernel_size = 5
filters = 64
pool_size = 4
    # LSTM
lstm_output_size = 70
    # Training
batch_size = 30
epochs = 2
loss = "binary_crossentropy"
optimizer = "adam"
eval_metrics = "accuracy"
##             ##

## Reproducibility ## 
random.seed(random_state)  # Python's seed
np.random.seed(random_state)  # Numpy's seed
tf.set_random_seed(random_state)  # Tensorflow's seed
##                 ##

In [19]:
## RTX GPU Memory BUG Fix & Must also be placed at the top of the code else it doesn't work ##
from keras.backend import tensorflow_backend as K
tf_config = tf.compat.v1.ConfigProto()
tf_config.gpu_options.allow_growth = True                     # dynamically grow the memory used on the GPU
#tf_config.gpu_options.per_process_gpu_memory_fraction = 0.9  # fraction of the GPU to be used
#tf_config.log_device_placement = True                        # to log device placement (on which device the operation ran)
sess = tf.compat.v1.Session(config=tf_config)
K.set_session(sess)                     # set this TensorFlow session as the default session for Keras
##                                                                                          ##     

## Tensorflow Verbosity Module ##
default_verbosity = tf.compat.v1.logging.get_verbosity()
tf.compat.v1.logging.set_verbosity(tensorflow_verbosity)
print(f"\n--CHANGED TENSORFLOW VERBOSITY FROM {default_verbosity/10:.0f} (default) TO {tf.compat.v1.logging.get_verbosity()/10:.0f}")
##                             ##

## Tensorflow GPU Information Module ##
if print_tensorflow_GPU_info == True:
    print(f"\n--AVAILABLE GPUS:")
    K._get_available_gpus()
    print(f"\n--NUM OF GPUs AVAILABLE: {len(tf.config.experimental.list_physical_devices('GPU'))}")
    print(f"\n--IS TF BUILT WITH CUDA: {tf.test.is_built_with_cuda()}")
    print(f"\n--IS GPU AVAILABLE: {tf.test.is_gpu_available()}")
##                                   ##  


--CHANGED TENSORFLOW VERBOSITY FROM 2 (default) TO 2

--AVAILABLE GPUS:

--NUM OF GPUs AVAILABLE: 1

--IS TF BUILT WITH CUDA: True

--IS GPU AVAILABLE: True


In [20]:
# Data Loading and Preprocessing
print(f"\nLoading data...")
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
print(f"{len(x_train)} train sequences")
print(f"{len(x_test)} test sequences")

print(f"Padding sequences (samples x time)")
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print(f"x_train shape: {x_train.shape}")
print(f"x_test shape: {x_test.shape}")


Loading data...
25000 train sequences
25000 test sequences
Padding sequences (samples x time)
x_train shape: (25000, 100)
x_test shape: (25000, 100)


In [21]:
# Model Building
print(f"\nBuilding model...")
model = Sequential()
model.add(Embedding(max_features, embedding_size, input_length=maxlen))
model.add(Dropout(dropout))
model.add(Conv1D(filters,
                 kernel_size,
                 padding='valid',
                 activation='relu',
                 strides=1))
model.add(MaxPooling1D(pool_size=pool_size))
model.add(LSTM(lstm_output_size))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss=loss,
              optimizer=optimizer,
              metrics=[eval_metrics])

early_stop = EarlyStopping(monitor='val_loss', mode='min', baseline=0.70, patience=3, verbose=1)
model_save = ModelCheckpoint('Saved Models/Introductory_text_imdb_LSTM.h5', monitor='val_loss', mode='min', save_best_only=True)



Building model...


In [22]:
# Model Training
print(f"Training...")
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    validation_data=(x_test, y_test),
                    callbacks=[early_stop, model_save],
                    verbose=1
                    )

Training...
Train on 25000 samples, validate on 25000 samples
Epoch 1/2
25000/25000 [==============================] - 23s 909us/step - loss: 0.3896 - accuracy: 0.8143 - val_loss: 0.3143 - val_accuracy: 0.8618
Epoch 2/2
25000/25000 [==============================] - 22s 895us/step - loss: 0.1939 - accuracy: 0.9257 - val_loss: 0.3401 - val_accuracy: 0.8563


In [23]:
# Model Evaluation
score, acc = model.evaluate(x_test, y_test, batch_size=batch_size)
print(f"Test loss: {score*100:.5f}")
print(f"Test accuracy: {acc*100:.5f}")

25000/25000 [==============================] - 8s 309us/step
Test loss: 34.01125
Test accuracy: 85.62800
